In [62]:
# == sys path ==
from pathlib import Path
import sys

project_root = Path().resolve().parent
sys.path.append(str(project_root))
# == sys path ==

import pandas as pd
import cv2
import numpy as np
from pathlib import Path
from typing import Dict, Iterable, Optional

LESION_MASKS = {
    'microaneurysms': 'Microaneurysms_Masks',
    'hemorrhages': 'Hemorrhage_Masks',
    'hard_exudates': 'HardExudate_Masks',
    'soft_exudates': 'SoftExudate_Masks',
}

LESION_LABELS = list(LESION_MASKS.keys())

# Default colors (BGR)
DEFAULT_COLOR_MAP = {
    "microaneurysms": (0, 0, 255),    # red
    "hemorrhages":    (255, 0, 0),    # blue
    "hard_exudates":  (0, 255, 255),  # yellow
    "soft_exudates":  (0, 255, 0),    # green
    "healthy":        (180, 180, 180) # gray
}

IMAGE_ID = r"0013_1"
IMAGE_DIR = r"/Users/jbalkovec/Desktop/DR/data/Seg-set/Original_Images"


DF_PATH = r'/Users/jbalkovec/Desktop/DR/data/patches/master_df.pkl'
MASK_MA = f"/Users/jbalkovec/Desktop/DR/data/Seg-set/Microaneurysms_Masks/{IMAGE_ID}.png"
MASK_HE = f"/Users/jbalkovec/Desktop/DR/data/Seg-set/Hemorrhage_Masks/{IMAGE_ID}.png"
MASK_SE = f"/Users/jbalkovec/Desktop/DR/data/Seg-set/SoftExudate_Masks/{IMAGE_ID}.png"
MASK_EX = f"/Users/jbalkovec/Desktop/DR/data/Seg-set/HardExudate_Masks/{IMAGE_ID}.png"
IMAGE =   f"/Users/jbalkovec/Desktop/DR/data/Seg-set/Original_Images/{IMAGE_ID}.png"

MASKS = {
    f"/Users/jbalkovec/Desktop/DR/data/Seg-set/Microaneurysms_Masks/{IMAGE_ID}.png": DEFAULT_COLOR_MAP["microaneurysms"],
    f"/Users/jbalkovec/Desktop/DR/data/Seg-set/Hemorrhage_Masks/{IMAGE_ID}.png":     DEFAULT_COLOR_MAP["hemorrhages"],
    f"/Users/jbalkovec/Desktop/DR/data/Seg-set/SoftExudate_Masks/{IMAGE_ID}.png":    DEFAULT_COLOR_MAP["soft_exudates"],
    f"/Users/jbalkovec/Desktop/DR/data/Seg-set/HardExudate_Masks/{IMAGE_ID}.png":    DEFAULT_COLOR_MAP["hard_exudates"],
}

In [63]:
def _class_from_label_vector(label_vector, lesion_labels) -> Optional[str]:
    if not isinstance(label_vector, (list, tuple)):
        return None
    if max(label_vector) == 0:
        return None
    idx = int(np.argmax(np.array(label_vector)))
    if 0 <= idx < len(lesion_labels):
        return lesion_labels[idx]
    return None

def _draw_x(img, center, size=12, color=(0, 255, 0), thickness=4):
    x, y = int(center["x"]), int(center["y"])
    cv2.line(img, (x - size, y - size), (x + size, y + size), color, thickness, cv2.LINE_AA)
    cv2.line(img, (x - size, y + size), (x + size, y - size), color, thickness, cv2.LINE_AA)

def _draw_rect(img, center, patch_size=128, color=(0, 255, 0), thickness=4):
    x, y = int(center["x"]), int(center["y"])
    half = patch_size // 2
    pt1 = (max(0, x - half), max(0, y - half))
    pt2 = (min(img.shape[1] - 1, x + half - 1), min(img.shape[0] - 1, y + half - 1))
    cv2.rectangle(img, pt1, pt2, color, thickness, cv2.LINE_AA)

def visual_check(
    image_path: Path,
    df_rows: Iterable[dict],
    out_path: Path,
    lesion_labels: Iterable[str],
    color_map: Dict[str, tuple] = DEFAULT_COLOR_MAP,
    patch_size: int = 128,
    x_size: int = 12,
    thickness: int = 4,   # default to bold lines
    alpha: float = 0.35
) -> Path:

    image_path = Path(image_path)
    out_path = Path(out_path)

    img_bgr = cv2.imread(str(image_path), cv2.IMREAD_COLOR)
    assert img_bgr is not None, f"Cannot read image: {image_path}"

    overlay = img_bgr.copy()

    for row in df_rows:
        center = row.get("center") or row.get("center_xy") or row.get("center_xy_dict")
        if not center or "x" not in center or "y" not in center:
            continue

        cls_name = _class_from_label_vector(row.get("label_vector"), lesion_labels)
        if cls_name is None:
            color = color_map.get("healthy", (180, 180, 180))
        else:
            color = color_map.get(cls_name, (0, 255, 0))

        _draw_rect(overlay, center, patch_size=patch_size, color=color, thickness=thickness)
        _draw_x(overlay, center, size=x_size, color=color, thickness=thickness)

    # Blend for visibility
    out = overlay

    out_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(out_path), out)
    return out_path

In [64]:
df = pd.read_pickle(DF_PATH)

In [65]:
df.head(n=5)

,image_id,patch_id,file_path,filter_tag,coordinates,center,label_vector
0,0000_1,0000_1_0751_0503,patches/0000_1/all/0000_1_0751_0503.png,lesion,"{'top-left': (751, 503), 'top-right': (878, 50...","{'x': 751, 'y': 503}","[1, 0, 0, 0]"
1,0000_1,0000_1_0450_0194,patches/0000_1/all/0000_1_0450_0194.png,healthy,"{'top-left': (450, 194), 'top-right': (577, 19...","{'x': 450, 'y': 194}","[0, 0, 0, 0]"
2,0000_1,0000_1_0278_0279,patches/0000_1/all/0000_1_0278_0279.png,healthy,"{'top-left': (278, 279), 'top-right': (405, 27...","{'x': 278, 'y': 279}","[0, 0, 0, 0]"
3,0000_1,0000_1_0539_0413,patches/0000_1/all/0000_1_0539_0413.png,healthy,"{'top-left': (539, 413), 'top-right': (666, 41...","{'x': 539, 'y': 413}","[0, 0, 0, 0]"
4,0000_1,0000_1_0897_0133,patches/0000_1/all/0000_1_0897_0133.png,healthy,"{'top-left': (897, 133), 'top-right': (1024, 1...","{'x': 897, 'y': 133}","[0, 0, 0, 0]"


In [66]:
  # change as needed
rows = df[df["image_id"] == IMAGE_ID].to_dict("records")

# Original image path
image_path = IMAGE_DIR  + "/" + f"{IMAGE_ID}.png"

# Output path
out_path = Path('/Users/jbalkovec/Desktop/DR/data/patches/visual_check') / f"{IMAGE_ID}_overlay.png"

# Draw
visual_check(
    image_path=image_path,
    df_rows=rows,
    out_path=out_path,
    lesion_labels=LESION_LABELS,
    color_map=DEFAULT_COLOR_MAP,
    patch_size=128,
    x_size=12,
    thickness=2,
    alpha=0.35
)

print(f"Saved overlay to: {out_path}")

Saved overlay to: /Users/jbalkovec/Desktop/DR/data/patches/visual_check/0013_1_overlay.png


In [67]:
def overlay_masks(image_path, mask_paths_to_color: dict, out_path, alpha=0.5, draw_contours=True):
    image = cv2.imread(str(image_path), cv2.IMREAD_COLOR)
    assert image is not None, f"Cannot read image: {image_path}"
    overlay = image.copy()

    for mask_path, color in mask_paths_to_color.items():
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue

        m = mask > 0
        if not m.any():
            continue

        # color layer only on masked pixels
        color_layer = np.zeros_like(overlay, dtype=np.uint8)
        for c in range(3):
            color_layer[:, :, c][m] = color[c]

        # blend only on masked pixels
        blended = cv2.addWeighted(overlay, 1.0, color_layer, alpha, 0.0)
        overlay[m] = blended[m]

        if draw_contours:
            contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(overlay, contours, -1, color, thickness=1, lineType=cv2.LINE_AA)

    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(out_path), overlay)
    return out_path

In [68]:
out = overlay_masks(
    image_path=IMAGE,
    mask_paths_to_color={
        MASK_MA: DEFAULT_COLOR_MAP["microaneurysms"],
        MASK_HE: DEFAULT_COLOR_MAP["hemorrhages"],
        MASK_SE: DEFAULT_COLOR_MAP["soft_exudates"],
        MASK_EX: DEFAULT_COLOR_MAP["hard_exudates"],
    },
    out_path=Path('/Users/jbalkovec/Desktop/DR/data/patches/visual_check') / f"{IMAGE_ID}_mask_overlay.png",
    alpha=0.5,
    draw_contours=True
)
print(f"Saved to: {out}")

Saved to: /Users/jbalkovec/Desktop/DR/data/patches/visual_check/0013_1_mask_overlay.png


In [69]:
def _class_from_label_vector(label_vector, lesion_labels) -> Optional[str]:
    if not isinstance(label_vector, (list, tuple)):
        return None
    if max(label_vector) == 0:
        return None
    idx = int(np.argmax(np.array(label_vector)))
    if 0 <= idx < len(lesion_labels):
        return lesion_labels[idx]
    return None

def _draw_x(img, center, size=12, color=(0, 255, 0), thickness=2):
    x, y = int(center["x"]), int(center["y"])
    cv2.line(img, (x - size, y - size), (x + size, y + size), color, thickness, cv2.LINE_AA)
    cv2.line(img, (x - size, y + size), (x + size, y - size), color, thickness, cv2.LINE_AA)

def _draw_rect(img, center, patch_size=128, color=(0, 255, 0), thickness=2):
    x, y = int(center["x"]), int(center["y"])
    half = patch_size // 2
    pt1 = (max(0, x - half), max(0, y - half))
    pt2 = (min(img.shape[1] - 1, x + half - 1), min(img.shape[0] - 1, y + half - 1))
    cv2.rectangle(img, pt1, pt2, color, thickness, cv2.LINE_AA)

def overlay_masks_and_patches(
    image_path: Path,
    mask_paths_to_color: Dict[str, tuple],
    df_rows: Iterable[dict],
    lesion_labels: Iterable[str],
    out_path: Path,
    patch_size: int = 128,
    alpha: float = 0.5,
    draw_contours: bool = True,
    x_size: int = 12,
    thickness: int = 2
) -> Path:
    image_path = Path(image_path)
    out_path = Path(out_path)

    img_bgr = cv2.imread(str(image_path), cv2.IMREAD_COLOR)
    assert img_bgr is not None, f"Cannot read image: {image_path}"
    H, W = img_bgr.shape[:2]

    overlay = img_bgr.copy()

    # --- Overlay lesion masks ---
    for mask_path, color in mask_paths_to_color.items():
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue
        if mask.shape[:2] != (H, W):
            mask = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)
        m = mask > 0
        if not m.any():
            continue

        color_layer = np.zeros_like(overlay, dtype=np.uint8)
        color_layer[m] = color

        blended = cv2.addWeighted(overlay, 1.0, color_layer, alpha, 0.0)
        overlay[m] = blended[m]

        if draw_contours:
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(overlay, contours, -1, color, thickness=1, lineType=cv2.LINE_AA)

    # --- Overlay patches ---
    for row in df_rows:
        center = row.get("center")
        if not center or "x" not in center or "y" not in center:
            continue
        cls_name = _class_from_label_vector(row.get("label_vector"), lesion_labels)
        if cls_name is None:
            color = DEFAULT_COLOR_MAP.get("healthy", (180, 180, 180))
        else:
            color = DEFAULT_COLOR_MAP.get(cls_name, (0, 255, 0))

        _draw_rect(overlay, center, patch_size=patch_size, color=color, thickness=thickness)
        _draw_x(overlay, center, size=x_size, color=color, thickness=thickness)

    out_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(out_path), overlay)
    return out_path

In [70]:
# Load DF and pick rows
df = pd.read_pickle("/Users/jbalkovec/Desktop/DR/data/patches/master_df.pkl")
rows = df[df["image_id"] == IMAGE_ID].to_dict("records")

# Run overlay
out = overlay_masks_and_patches(
    image_path=IMAGE,
    mask_paths_to_color=MASKS,
    df_rows=rows,
    lesion_labels=LESION_LABELS,
    out_path=Path(f"/Users/jbalkovec/Desktop/DR/data/patches/visual_check/{IMAGE_ID}combined.png"),
    patch_size=128,
    alpha=0.5
)

print(f"Saved combined overlay to {out}")

Saved combined overlay to /Users/jbalkovec/Desktop/DR/data/patches/visual_check/0013_1combined.png


In [71]:
lesion_df = df[(df["image_id"] == IMAGE_ID) & (df["filter_tag"] != "healthy")]

print(f"{IMAGE_ID} has {len(lesion_df)} non-healthy patches")

0013_1 has 11 non-healthy patches


In [72]:
lesion_df.head(n=11)

,image_id,patch_id,file_path,filter_tag,coordinates,center,label_vector
31,0013_1,0013_1_0935_1115,patches/0013_1/all/0013_1_0935_1115.png,lesion,"{'top-left': (935, 1115), 'top-right': (1062, ...","{'x': 935, 'y': 1115}","[1, 0, 0, 0]"
33,0013_1,0013_1_1000_0621,patches/0013_1/all/0013_1_1000_0621.png,lesion,"{'top-left': (1000, 621), 'top-right': (1127, ...","{'x': 1000, 'y': 621}","[1, 0, 0, 0]"
34,0013_1,0013_1_1081_0774,patches/0013_1/all/0013_1_1081_0774.png,lesion,"{'top-left': (1081, 774), 'top-right': (1208, ...","{'x': 1081, 'y': 774}","[1, 0, 0, 0]"
35,0013_1,0013_1_0772_0657,patches/0013_1/all/0013_1_0772_0657.png,lesion,"{'top-left': (772, 657), 'top-right': (899, 65...","{'x': 772, 'y': 657}","[0, 0, 1, 0]"
36,0013_1,0013_1_0796_1204,patches/0013_1/all/0013_1_0796_1204.png,lesion,"{'top-left': (796, 1204), 'top-right': (923, 1...","{'x': 796, 'y': 1204}","[1, 0, 0, 0]"
37,0013_1,0013_1_0911_1104,patches/0013_1/all/0013_1_0911_1104.png,lesion,"{'top-left': (911, 1104), 'top-right': (1038, ...","{'x': 911, 'y': 1104}","[1, 0, 0, 0]"
38,0013_1,0013_1_0685_1019,patches/0013_1/all/0013_1_0685_1019.png,lesion,"{'top-left': (685, 1019), 'top-right': (812, 1...","{'x': 685, 'y': 1019}","[1, 0, 0, 0]"
39,0013_1,0013_1_1067_0678,patches/0013_1/all/0013_1_1067_0678.png,lesion,"{'top-left': (1067, 678), 'top-right': (1194, ...","{'x': 1067, 'y': 678}","[1, 0, 0, 0]"
40,0013_1,0013_1_1049_0649,patches/0013_1/all/0013_1_1049_0649.png,lesion,"{'top-left': (1049, 649), 'top-right': (1176, ...","{'x': 1049, 'y': 649}","[1, 0, 0, 0]"
41,0013_1,0013_1_0990_0583,patches/0013_1/all/0013_1_0990_0583.png,lesion,"{'top-left': (990, 583), 'top-right': (1117, 5...","{'x': 990, 'y': 583}","[1, 0, 0, 0]"
